In [5]:
from surprise import SVDpp, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
import sigopt

# Замените 'YOUR_API_TOKEN' на ваш фактический API-токен SigOpt
conn = sigopt.Connection(client_token='WYKGPMSPGMXTRNJMHSNLQYXOYEJMBUXTYQDZVAKWZNVOTLQO')

# Load the movielens-100k dataset
data = Dataset.load_builtin('ml-100k')

# Define the trainset and testset
trainset, testset = train_test_split(data, test_size=.25)

# Define the parameter bounds
param_bounds = {
    'n_epochs': [5, 20],
    'lr_all': [0.001, 0.01],
    'reg_all': [0.01, 0.1]
}

# Create an experiment on SigOpt
experiment_svdpp = conn.experiments().create(
    name='SVDpp Recommender',
    parameters=[
        dict(name='n_epochs', type='int', bounds=dict(min=5, max=20)),
        dict(name='lr_all', type='double', bounds=dict(min=0.001, max=0.01)),
        dict(name='reg_all', type='double', bounds=dict(min=0.01, max=0.1))
    ],
    metrics=[
        dict(name='RMSE', objective='minimize', strategy='optimize'),
        dict(name='MAE', objective='minimize', strategy='store')
    ],
    parallel_bandwidth=1,
    observation_budget=50,
)

print("Created experiment: https://app.sigopt.com/experiment/" + experiment_svdpp.id)


# Evaluate your model with the suggested parameter assignments
def create_model_svdpp(assignments):
    algo_svdpp = SVDpp(
        n_epochs=assignments['n_epochs'],
        lr_all=assignments['lr_all'],
        reg_all=assignments['reg_all']
    ).fit(trainset)
    return algo_svdpp

def evaluate_model_svdpp(assignments):
    algo_svdpp = create_model_svdpp(assignments)
    predictions_svdpp = algo_svdpp.test(testset)
    # Compute RMSE and MAE:
    return [
        dict(name="RMSE", value=accuracy.rmse(predictions_svdpp)),
        dict(name="MAE", value=accuracy.mae(predictions_svdpp))
    ]


# Run the Optimization Loop until the Observation Budget is exhausted
while experiment_svdpp.progress.observation_count < experiment_svdpp.observation_budget:
    suggestion_svdpp = conn.experiments(experiment_svdpp.id).suggestions().create()
    value_dicts_svdpp = evaluate_model_svdpp(suggestion_svdpp.assignments)
    conn.experiments(experiment_svdpp.id).observations().create(
        suggestion=suggestion_svdpp.id,
        values=value_dicts_svdpp,
    )

    # Update the experiment object
    experiment_svdpp = conn.experiments(experiment_svdpp.id).fetch()

# Fetch the best configuration and explore your experiment
all_best_assignments_svdpp = conn.experiments(experiment_svdpp.id).best_assignments().fetch()
# Returns a list of dict-like Observation objects
best_assignments_svdpp = all_best_assignments_svdpp.data[0].assignments
print("Best Assignments (SVDpp): " + str(best_assignments_svdpp))
print("Explore your experiment: https://app.sigopt.com/experiment/" + experiment_svdpp.id + "/analysis")


Created experiment: https://app.sigopt.com/experiment/1024556
RMSE: 0.9532
MAE:  0.7568
RMSE: 0.9353
MAE:  0.7386
RMSE: 0.9352
MAE:  0.7325
RMSE: 0.9594
MAE:  0.7616
RMSE: 0.9252
MAE:  0.7274
RMSE: 0.9236
MAE:  0.7273
RMSE: 0.9229
MAE:  0.7281
RMSE: 0.9264
MAE:  0.7309
RMSE: 0.9202
MAE:  0.7250
RMSE: 0.9222
MAE:  0.7258
RMSE: 0.9329
MAE:  0.7362
RMSE: 0.9209
MAE:  0.7260
RMSE: 0.9252
MAE:  0.7267
RMSE: 0.9671
MAE:  0.7685
RMSE: 0.9272
MAE:  0.7307
RMSE: 0.9383
MAE:  0.7400
RMSE: 0.9264
MAE:  0.7250
RMSE: 0.9475
MAE:  0.7503
RMSE: 0.9259
MAE:  0.7299
RMSE: 0.9266
MAE:  0.7330
RMSE: 0.9230
MAE:  0.7293
RMSE: 0.9496
MAE:  0.7527
RMSE: 0.9227
MAE:  0.7281
RMSE: 0.9241
MAE:  0.7284
RMSE: 0.9326
MAE:  0.7380
RMSE: 0.9335
MAE:  0.7369
RMSE: 0.9352
MAE:  0.7402
RMSE: 0.9345
MAE:  0.7367
RMSE: 0.9231
MAE:  0.7251
RMSE: 0.9240
MAE:  0.7303
RMSE: 0.9244
MAE:  0.7285
RMSE: 0.9258
MAE:  0.7315
RMSE: 0.9251
MAE:  0.7289
RMSE: 0.9221
MAE:  0.7276
RMSE: 0.9451
MAE:  0.7472
RMSE: 0.9243
MAE:  0.7262
RM

c:\Users\grinu\AppData\Local\Programs\Python\Python38\lib\site-packages\sigopt\objects.py:338: RuntimeWarning: The .data field only contains a single page of results, which may be incomplete for large responses. Prefer to use the `.iterate_pages() to ensure that you iterate through all elements in the response.
  warnings.warn(
